In [43]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

data_uni_2 = pd.read_csv("data_uni.csv")

data_uni_2.iloc[:, [24,25,29]]= data_uni_2.iloc[:, [24,25,29]].fillna(0)
column_miss= list(data_uni_2.columns[data_uni_2.isnull().any()])

# IMPUTER ===========================================================================

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

data_uni_2[column_miss]= imputer.fit_transform(data_uni_2[column_miss])

data_uni_2= data_uni_2.drop(["Course","AWM year 1", "AWM year 2", "AWM year 3", "Overall AWM", "Progression", "Attendance"], axis = 1)

# FORMATING MANUALLY ================================================================

#ethinicty binarisation 
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("White", "0")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Asian", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Black/Black British African", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Other ethnic background", "1")

# Polar 4 Score Binarisation
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(5, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(4, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(3, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(2, 1)

#splitting X and y ==================================================================
y = data_uni_2["desertion"]
X = data_uni_2.drop(["desertion"], axis = 1)

In [44]:
#Test 3
from pycaret.classification import *
classify_3 = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance= False, feature_selection = True, session_id= 4670, normalize = False, pca= False, pca_components=0.9, numeric_features=["First Sit", "Fails","No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

,Description,Value
0,session_id,4670
1,Target,desertion
2,Target Type,Binary
3,Label Encoded,"no: 0, yes: 1"
4,Original Data,"(1033, 29)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,19
8,Ordinal Features,False
9,High Cardinality Features,False


In [45]:
compare_models(include=["rf", "ada","gbc"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9322,0.9573,0.7359,0.8439,0.7778,0.7388,0.7461,0.4070
ada,Ada Boost Classifier,0.9225,0.9335,0.7103,0.8105,0.7442,0.7001,0.7096,0.0520
gbc,Gradient Boosting Classifier,0.9183,0.9496,0.7269,0.7678,0.7424,0.6945,0.6975,0.0680


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=4670, verbose=0,
                       warm_start=False)

In [47]:
rf_3= create_model("rf", cross_validation= True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9918,0.8333,0.9091,0.8696,0.8452,0.8464
1,0.9315,0.9652,0.8333,0.7692,0.8000,0.7588,0.7596
2,0.9452,0.9481,0.6923,1.0000,0.8182,0.7872,0.8056
3,0.9444,0.9861,0.8333,0.8333,0.8333,0.8000,0.8000
4,0.8889,0.9104,0.4167,0.8333,0.5556,0.5000,0.5394
5,0.9167,0.9549,0.7500,0.7500,0.7500,0.7000,0.7000
6,0.9444,0.9528,0.7500,0.9000,0.8182,0.7857,0.7903
7,0.9444,0.9215,0.8333,0.8333,0.8333,0.8000,0.8000
8,0.9444,0.9799,0.8333,0.8333,0.8333,0.8000,0.8000


In [48]:
rf_3= create_model("rf", cross_validation= False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9387,0.9254,0.7297,0.75,0.7397,0.705,0.7051


In [49]:
tuned_rf_3= tune_model(rf_3, optimize = "Recall")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9863,0.9891,1.0000,0.9231,0.9600,0.9518,0.9529
1,0.9178,0.9658,0.9167,0.6875,0.7857,0.7361,0.7478
2,0.9315,0.9487,0.7692,0.8333,0.8000,0.7588,0.7596
3,0.9583,0.9847,0.9167,0.8462,0.8800,0.8548,0.8558
4,0.8750,0.9042,0.5833,0.6364,0.6087,0.5345,0.5352
5,0.9028,0.9514,0.9167,0.6471,0.7586,0.7000,0.7166
6,0.9306,0.9625,0.7500,0.8182,0.7826,0.7414,0.7424
7,0.9306,0.9319,0.8333,0.7692,0.8000,0.7581,0.7590
8,0.9028,0.9750,0.8333,0.6667,0.7407,0.6818,0.6882


In [50]:
final_rf_3 = calibrate_model(tuned_rf_3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9891,0.8333,0.9091,0.8696,0.8452,0.8464
1,0.9178,0.9658,0.8333,0.7143,0.7692,0.7196,0.7228
2,0.9315,0.9551,0.6923,0.9000,0.7826,0.7428,0.7518
3,0.9583,0.9847,0.8333,0.9091,0.8696,0.8448,0.8460
4,0.9028,0.9056,0.5833,0.7778,0.6667,0.6111,0.6198
5,0.8889,0.9542,0.8333,0.6250,0.7143,0.6471,0.6574
6,0.9444,0.9625,0.7500,0.9000,0.8182,0.7857,0.7903
7,0.9306,0.9236,0.6667,0.8889,0.7619,0.7222,0.7325
8,0.9306,0.9764,0.7500,0.8182,0.7826,0.7414,0.7424


In [51]:
evaluate_model(final_rf_3)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [52]:
gbc_3 = create_model("gbc",cross_validation= False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9323,0.9363,0.7838,0.6905,0.7342,0.6955,0.6974


In [53]:
gbc_3 = create_model("gbc",cross_validation= True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9945,0.8333,0.9091,0.8696,0.8452,0.8464
1,0.9315,0.9672,0.8333,0.7692,0.8000,0.7588,0.7596
2,0.9315,0.9513,0.7692,0.8333,0.8000,0.7588,0.7596
3,0.9583,0.9889,0.8333,0.9091,0.8696,0.8448,0.8460
4,0.8333,0.8722,0.3333,0.5000,0.4000,0.3077,0.3162
5,0.8889,0.9319,0.7500,0.6429,0.6923,0.6250,0.6278
6,0.9444,0.9389,0.7500,0.9000,0.8182,0.7857,0.7903
7,0.9444,0.9236,0.8333,0.8333,0.8333,0.8000,0.8000
8,0.9167,0.9736,0.8333,0.7143,0.7692,0.7188,0.7219


In [54]:
tuned_gbc_3= tune_model(gbc_3, optimize = "Recall")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9726,0.9891,0.9167,0.9167,0.9167,0.9003,0.9003
1,0.9589,0.9781,0.8333,0.9091,0.8696,0.8452,0.8464
2,0.9589,0.9603,0.8462,0.9167,0.8800,0.8553,0.8563
3,0.9722,0.9917,0.9167,0.9167,0.9167,0.9000,0.9000
4,0.8750,0.9208,0.3333,0.8000,0.4706,0.4130,0.4642
5,0.9028,0.9556,0.7500,0.6923,0.7200,0.6613,0.6621
6,0.9167,0.9639,0.6667,0.8000,0.7273,0.6786,0.6825
7,0.9583,0.9278,0.8333,0.9091,0.8696,0.8448,0.8460
8,0.9444,0.9806,0.9167,0.7857,0.8462,0.8125,0.8161


In [55]:
final_gbc_3 = calibrate_model(tuned_gbc_3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9850,0.8333,0.9091,0.8696,0.8452,0.8464
1,0.9452,0.9754,0.8333,0.8333,0.8333,0.8005,0.8005
2,0.9315,0.9474,0.6923,0.9000,0.7826,0.7428,0.7518
3,0.9583,0.9889,0.8333,0.9091,0.8696,0.8448,0.8460
4,0.8611,0.9153,0.3333,0.6667,0.4444,0.3750,0.4045
5,0.9167,0.9528,0.7500,0.7500,0.7500,0.7000,0.7000
6,0.9306,0.9514,0.6667,0.8889,0.7619,0.7222,0.7325
7,0.9444,0.9208,0.7500,0.9000,0.8182,0.7857,0.7903
8,0.9444,0.9819,0.9167,0.7857,0.8462,0.8125,0.8161


In [56]:
evaluate_model(final_gbc_3)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [57]:
ada_3 = create_model("ada",cross_validation= False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9484,0.9342,0.7297,0.8182,0.7714,0.7424,0.744


In [58]:
ada_3 = create_model("ada",cross_validation= True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9315,0.9686,0.6667,0.8889,0.7619,0.7229,0.7331
1,0.9452,0.9467,0.8333,0.8333,0.8333,0.8005,0.8005
2,0.9315,0.9115,0.7692,0.8333,0.8000,0.7588,0.7596
3,0.9444,0.9681,0.7500,0.9000,0.8182,0.7857,0.7903
4,0.8750,0.8611,0.3333,0.8000,0.4706,0.4130,0.4642
5,0.9028,0.9458,0.7500,0.6923,0.7200,0.6613,0.6621
6,0.9306,0.9458,0.6667,0.8889,0.7619,0.7222,0.7325
7,0.9306,0.8958,0.7500,0.8182,0.7826,0.7414,0.7424
8,0.9444,0.9681,1.0000,0.7500,0.8571,0.8235,0.8367


In [59]:
tuned_ada_3= tune_model(ada_3, optimize = "Recall")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9315,0.9727,0.6667,0.8889,0.7619,0.7229,0.7331
1,0.9589,0.9474,0.8333,0.9091,0.8696,0.8452,0.8464
2,0.9178,0.9077,0.6923,0.8182,0.7500,0.7012,0.7047
3,0.9444,0.9778,0.7500,0.9000,0.8182,0.7857,0.7903
4,0.8750,0.8444,0.3333,0.8000,0.4706,0.4130,0.4642
5,0.9167,0.9486,0.7500,0.7500,0.7500,0.7000,0.7000
6,0.9306,0.9556,0.6667,0.8889,0.7619,0.7222,0.7325
7,0.9583,0.9069,0.8333,0.9091,0.8696,0.8448,0.8460
8,0.9028,0.9681,0.7500,0.6923,0.7200,0.6613,0.6621


In [60]:
final_ada_3 = calibrate_model(tuned_ada_3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9041,0.9822,0.5000,0.8571,0.6316,0.5808,0.6087
1,0.9589,0.9467,0.8333,0.9091,0.8696,0.8452,0.8464
2,0.9178,0.9154,0.5385,1.0000,0.7000,0.6573,0.6997
3,0.9444,0.9778,0.7500,0.9000,0.8182,0.7857,0.7903
4,0.8750,0.9167,0.3333,0.8000,0.4706,0.4130,0.4642
5,0.9167,0.9514,0.7500,0.7500,0.7500,0.7000,0.7000
6,0.9306,0.9625,0.6667,0.8889,0.7619,0.7222,0.7325
7,0.9444,0.9014,0.6667,1.0000,0.8000,0.7692,0.7906
8,0.9167,0.9750,0.6667,0.8000,0.7273,0.6786,0.6825


In [61]:
evaluate_model(final_ada_3)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…